In [0]:
! apt install python python-pip build-essential cmake pkg-config libx11-dev libatlas-base-dev libgtk-3-dev libboost-python-dev -y
!pip install dlib
! pip3 install face_recognition

In [0]:
# check for dlib on gpu
import dlib.cuda as cuda
print(cuda.get_num_devices())

1


In [3]:
import face_recognition
import cv2
import glob
import numpy as np
import os

In [0]:
files_path = "./face_recognition/"

In [0]:
# Open the input movie file
input_movie = cv2.VideoCapture(files_path+"input.mp4")

length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(input_movie.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_movie.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = input_movie.get(cv2.CAP_PROP_FPS)
frame_rate = round(frame_rate,2)

In [0]:
# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter(files_path+'output.avi', fourcc, frame_rate, (frame_width, frame_height))

In [4]:
# Load some sample pictures and learn how to recognize them.
# lmm_image = face_recognition.load_image_file("lin-manuel-miranda.png")
# lmm_face_encoding = face_recognition.face_encodings(lmm_image)[0]

# al_image = face_recognition.load_image_file("alex-lacamoire.png")
# al_face_encoding = face_recognition.face_encodings(al_image)[0]

known_faces = []
names = []
for image in sorted(glob.glob(files_path+'/*.jpg')):
    print(image)
    names.append(os.path.basename(image).split('.')[0])
    image_features = face_recognition.load_image_file(image)
    al_face_encoding = face_recognition.face_encodings(image_features)[0]
    known_faces.append(al_face_encoding)

In [0]:
%time
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0
dected_frames = []
while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame,model="cnn") # model ='cnn' uses gpu model = 'HOG' on cpu
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

        indexs = np.nonzero(match)[0]

        if len(indexs) != 0:

            for index in indexs:
                face_names.append(names[index])
        else:
            face_names.append(None)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

# All done!
input_movie.release()
output_movie.release()
cv2.destroyAllWindows()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
Writing frame 1 / 4907
Writing frame 2 / 4907
Writing frame 3 / 4907
Writing frame 4 / 4907
Writing frame 5 / 4907
Writing frame 6 / 4907
Writing frame 7 / 4907
Writing frame 8 / 4907
Writing frame 9 / 4907
Writing frame 10 / 4907
Writing frame 11 / 4907
Writing frame 12 / 4907
Writing frame 13 / 4907
Writing frame 14 / 4907
Writing frame 15 / 4907
Writing frame 16 / 4907
Writing frame 17 / 4907
Writing frame 18 / 4907
Writing frame 19 / 4907
Writing frame 20 / 4907
Writing frame 21 / 4907
Writing frame 22 / 4907
Writing frame 23 / 4907
Writing frame 24 / 4907
Writing frame 25 / 4907
Writing frame 26 / 4907
Writing frame 27 / 4907
Writing frame 28 / 4907
Writing frame 29 / 4907
Writing frame 30 / 4907
Writing frame 31 / 4907
Writing frame 32 / 4907
Writing frame 33 / 4907
Writing frame 34 / 4907
Writing frame 35 / 4907
Writing frame 36 / 4907
Writing frame 37 / 4907
Writing frame 38 / 4907
Writing frame 39 / 4907
Writing f

## Reference
- Face Recognition [The world's simplest facial recognition api for Python and the command line](https://github.com/ageitgey/face_recognition)